In [1]:
import pandas as pd

In [61]:
data= "data/CCLE/mutation_ccle.csv"
df = pd.read_csv(data, index_col=False)
df.shape

(264, 563)

In [65]:
cell_name=list(df.iloc[:,0])

cell_name


['A2058',
 'C32',
 'HT1197',
 'MDAMB157',
 'HT29',
 'AU565',
 'NCIH441',
 'SW1088',
 'HCT116',
 'KYSE450',
 'NCIH28',
 'NCIH358',
 'NCIH727',
 'NCIH1299',
 'SAOS2',
 'LS411N',
 'RKO',
 'RPMI7951',
 'WM115',
 'NCIH2009',
 'NCIH2452',
 'UACC62',
 'UACC257',
 'KELLY',
 'NCIH661',
 'MPP89',
 'NCIH2030',
 'COLO205',
 'REH',
 'NCIH1792',
 'NCIH1355',
 'NCIH1048',
 'OE33',
 'OVCAR8',
 'PANC1005',
 'NCIH1915',
 'OCIAML2',
 'ONS76',
 'RT112',
 'NCIH1793',
 'RERFGC1B',
 'RVH421',
 'PC14',
 'NCIH1581',
 'OPM2',
 'NCIH1092',
 'NCIH1573',
 'RKN',
 'PSN1',
 'NCIH1648',
 'OV90',
 'NCIH1568',
 'NCIH1944',
 'RD',
 'PANC0327',
 'OC314',
 'NCIH1155',
 'OE21',
 'KYSE520',
 'KURAMOCHI',
 'LOUNH91',
 'GAMG',
 'MFE319',
 'KLE',
 'LCLC103H',
 'KYSE410',
 'KP3',
 'L363',
 'LN229',
 'FADU',
 'G401',
 'GI1',
 'KYSE510',
 'KU812',
 'KCL22',
 'LN18',
 'FU97',
 'GCT',
 'MFE280',
 'KMRC1',
 'LS513',
 'KYSE70',
 'KYSE140',
 'KALS1',
 'LS123',
 'GCIY',
 'G402',
 'MELHO',
 'KYSE150',
 'GB1',
 'U118MG',
 'TE1',
 'SCC9',

In [57]:
print(df.columns)

Index(['Unnamed: 0',
       '('cell line: LN18', 'tissue: central_nervous_system', 'histology: glioma', 'sub-histology: astrocytoma_Grade_IV', 'gender: M')',
       '('cell line: 769P', 'tissue: kidney', 'histology: carcinoma', 'sub-histology: clear_cell_renal_cell_carcinoma', 'gender: F')',
       '('cell line: 786O', 'tissue: kidney', 'histology: carcinoma', 'sub-histology: clear_cell_renal_cell_carcinoma', 'gender: M')',
       '('cell line: CAOV3', 'tissue: ovary', 'histology: carcinoma', 'sub-histology: adenocarcinoma', 'gender: F')',
       '('cell line: HEPG2', 'tissue: liver', 'histology: carcinoma', 'sub-histology: hepatocellular_carcinoma', 'gender: M')',
       '('cell line: MOLT4', 'tissue: haematopoietic_and_lymphoid_tissue', 'histology: lymphoid_neoplasm', 'sub-histology: acute_lymphoblastic_T_cell_leukaemia', 'gender: M')',
       '('cell line: NCIH524', 'tissue: lung', 'histology: carcinoma', 'sub-histology: small_cell_carcinoma', 'gender: M')',
       '('cell line: NCI

In [15]:
import requests

def get_drug_smiles(drug_id):
    """
    Retrieve the SMILES string for a given drug ID using the PubChem API.
    """
    # Construct the API URL
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{drug_id}/property/CanonicalSMILES/json"

    # Send the API request
    response = requests.get(url)

    # Parse the JSON response
    json_data = response.json()

    # Extract the SMILES string
    smiles = json_data["PropertyTable"]["Properties"][0]["CanonicalSMILES"]

    return smiles


In [16]:
smiles = get_pubchem_smiles(9826528)
print(smiles)


C1=CC(=C(C=C1I)F)NC2=C(C=CC(=C2F)F)C(=O)NOCC(CO)O


In [17]:

import requests
import time


In [12]:
def execute(name):

    drug_names = []
    drug_names.append(name)

    base_url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug"

    url_for_synonyms_to_cid = f"{base_url}/compound/name/synonyms/json"
    response_pub_chem_synonyms = requests.post(url_for_synonyms_to_cid, {"name": name})
    if response_pub_chem_synonyms.status_code > 202:
        raise Exception(f"PUBChem ({url_for_synonyms_to_cid})\nResponse code {response_pub_chem_synonyms.status_code}\n"
                        f"{response_pub_chem_synonyms.text}")
    response_pub_chem_synonyms = response_pub_chem_synonyms.json()
    cid_record = response_pub_chem_synonyms["InformationList"]["Information"][0]

    time.sleep(1)

    compound_properties = {}
    url_for_pub_chem_smiles_iupac = f"{base_url}/compound/cid/property/IUPACName,CanonicalSMILES/json"
    response_pub_chem_smiles_iupac = requests.post(url_for_pub_chem_smiles_iupac, {"cid": cid_record["CID"]})
    if response_pub_chem_smiles_iupac.status_code > 202:
        raise Exception(f"PUBChem ({url_for_pub_chem_smiles_iupac})\n"
                        f"Response code {response_pub_chem_smiles_iupac.status_code}\n"
                        f"{response_pub_chem_smiles_iupac.text}")
    response_pub_chem_smiles_iupac = response_pub_chem_smiles_iupac.json()
    for params in response_pub_chem_smiles_iupac["PropertyTable"]["Properties"]:
        try:
            compound_properties[params["CID"]] = [params["CanonicalSMILES"], params["IUPACName"],
                                                  cid_record["Synonym"][0]]
        except KeyError:
            compound_properties[params["CID"]] = ["", "", ""]
    SMILES= list(compound_properties.values())[0][0]
    return SMILES

In [13]:
a=execute("9826528")
a

Exception: PUBChem (https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/synonyms/json)
Response code 404
{
  "Fault": {
    "Code": "PUGREST.NotFound",
    "Message": "No CID found",
    "Details": [
      "No CID found that matches the given name"
    ]
  }
}


In [14]:
print(a)

NameError: name 'a' is not defined

In [68]:
import numpy as np
import pandas as pd

pd.read_excel("data/CCLE/drug_id_name.xlsx", header=None)

,0,1,2
0,1,11476171,AEW541
1,2,644241,Nilotinib
2,3,6505803,17-AAG
3,4,10461815,PHA-665752
4,5,208908,Lapatinib
5,6,216345,Nutlin-3
6,7,10302451,AZD0530
7,8,11626560,PF2341066
8,9,5479543,L-685458
9,10,3081361,ZD-6474


In [70]:
df=pd.read_csv("data/CCLE/drugResponse_ccle.csv", index_col=0)
df

,AEW541,Nilotinib,X17.AAG,PHA.665752,Lapatinib,Nutlin.3,AZD0530,PF2341066,L.685458,ZD.6474,...,LBW242,PD.0325901,PD.0332991,Paclitaxel,AZD6244,PLX4720,RAF265,TAE684,TKI258,Erlotinib
A2058,1.0006,0.02046,4.7470,0.200300,0.12020,0.1101,0.7930,0.5221,0.06193,1.3466,...,1.19890,2.4465,0.00000,7.1679,1.8154,0.7129,0.9206,0.8825,0.8580,0.00000
C32,0.8161,0.13930,6.5863,0.191400,0.00000,0.0000,0.0000,0.2610,0.04038,0.4507,...,0.49080,3.2117,0.07469,3.1892,1.8523,1.2483,1.1826,0.6679,0.3908,0.28600
HT1197,1.5118,0.06098,1.8151,0.008045,0.35320,0.3538,0.7165,0.2529,0.04266,1.0391,...,0.35800,2.0249,0.00000,1.9670,1.1711,0.0000,1.8252,1.0320,0.6075,0.26420
MDAMB157,0.7972,0.65890,0.8792,0.014940,0.11380,0.4724,0.5788,1.1257,0.42680,0.4384,...,1.11440,0.7510,0.11570,3.8675,0.6353,0.0486,1.9967,1.1916,1.4616,0.26840
HT29,1.8932,0.43980,5.3135,0.264800,0.80290,0.3962,0.9439,0.5708,0.14030,1.2904,...,0.17590,4.6830,0.34300,6.2246,4.1480,1.6266,2.0208,2.1444,0.9653,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NCIH2122,1.7100,0.57320,4.9960,0.792200,1.05450,0.3708,0.8178,0.6180,0.34960,0.8166,...,0.91730,4.4466,1.27400,6.6492,3.0327,0.8008,2.0903,1.8716,1.0366,0.89590
MFE296,0.9253,NaN,2.9064,0.453000,0.44470,0.4547,1.1319,0.5073,0.48720,1.5118,...,1.02660,0.6229,NaN,5.9629,0.3140,0.3759,1.9443,0.7959,1.3124,0.61760
NCIH226,0.6322,0.00000,2.0162,0.000000,0.00000,0.1363,0.2167,0.1941,0.00000,0.2921,...,0.04654,1.5890,0.26010,3.2070,0.3049,0.0000,1.3318,0.7534,0.8242,0.00000
SU8686,0.7582,NaN,2.4711,0.285400,0.09048,0.1480,0.3333,0.2915,0.01013,0.5285,...,0.90710,0.7627,0.00000,4.1376,0.6160,0.2061,1.1501,0.2885,0.1613,0.18660


In [71]:
tmp=[]
for idx,row in df.iterrows():
  for col in df.columns:
    tmp.append((col,idx,df.loc[idx,col]))

In [73]:
print(tmp[5])

('Nutlin.3', 'A2058', 0.1101)
